<a href="https://colab.research.google.com/github/nicolerdz/Regresion-no-lineal-y-logistica/blob/main/2021__9_Regresion_Logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#Cargar archivo csv desde seaborn
df=pd.read_csv('2021Datos_limpios.csv', index_col= 0)
df.head(20)

,FECHA,FOLIO,UUID,RFC,PROVEEDOR,TIPO GASTO,DESCRIPCION,MP,TC,TIPO,STATUS,POLIZA,IMPORTE,IVA,RET ISR,RET IVA,TOTAL MX,TOTAL SAT
0,2021-01-01,0001010100750847116902,71B4C8B9-8DAA-47CC-AD47-9DD90AA5AF6B,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,COMISION COBRADA,PUE,1.0,I,Vigente,0,490.00,78.40,0.0,0.0,568.4,568.40
1,2021-01-01,0001010100752163714102,B8AEDE83-CC76-404C-A611-6B5A87BA561F,BNM840515VB1,"BANCO NACIONAL DE MEXICO, S.A.",COMISION BANCARIA,COMISION COBRADA,PUE,1.0,I,Vigente,0,345.00,55.20,0.0,0.0,400.2,400.20
2,2021-01-01,19788,19B9B2C2-DF58-E24D-B43A-3FC4D5A929E7,ESI9612311E7,"ELECTRONICOS Y SIMILARES, S.A. DE C.V.",COMPRAS,MOTO E7 PLUS AZUL 64GB 4RAM,PUE,1.0,I,Vigente,0,3447.41,551.59,0.0,0.0,3999.0,3999.00
3,2021-01-01,211781,EAB26414-DD9A-4E8F-9F5C-05FE24D98E48,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006876 6/36 de 03/12/2...,PPD,1.0,I,Vigente,0,5551.38,888.22,0.0,0.0,6439.6,6439.60
4,2021-01-01,211782,C16F79B5-FB35-4BFC-AF84-708B24F500BB,BLE1512082R9,BEPENSA LEASING S.A. DE C.V.,ARRENDAMIENTO FINANCIERO,AA RENTA EXIGIBLE CL0000006877 6/36 de 03/12/2...,PPD,1.0,I,Vigente,0,5551.98,888.32,0.0,0.0,6440.3,6440.30
5,2021-01-01,4101,1BDD6B05-B42C-46D8-AF16-0D91FEDCE822,ENA7006081P0,ELECTROTERMICA NACIONAL SA DE CV,COMPRAS,Pago,PUE,1.0,P,Vigente,0,0.00,0.00,0.0,0.0,0.0,0.00
6,2021-01-01,32322,60B3F20E-5A5E-4439-83C6-2F8052F90D2A,AAE6411055P8,"AESA ARTICULOS ELECTRICOS, S.A. DE C.V.",COMPRAS,Pago,PUE,1.0,P,Vigente,0,0.00,0.00,0.0,0.0,0.0,0.00
7,2021-01-01,9231013,26928ED7-4CBE-4102-99A5-DE5CF2D92698,ANE140618P37,Servicios Comerciales Amazon México S. de R.L....,NO DEDUCIBLE,"Pagani, Coche de Control Remoto Escala 1:24, C...",PUE,1.0,I,Vigente,0,0.00,0.00,0.0,0.0,0.0,599.00
8,2021-01-01,9231022,E6883E6F-6947-41DC-AF28-7027615DF8B4,ANE140618P37,Servicios Comerciales Amazon México S. de R.L....,NO DEDUCIBLE,Strathmore Libreta para Aprender a Dibujar Ani...,PUE,1.0,I,Vigente,0,0.00,0.00,0.0,0.0,0.0,335.37
9,2021-01-01,9231026,A54AB33B-1D51-4213-8147-A001D3980B72,ANE140618P37,Servicios Comerciales Amazon México S. de R.L....,NO DEDUCIBLE,ENCORDADURA DADDARIO MOD. J-810 4/4M,PUE,1.0,I,Vigente,0,0.00,0.00,0.0,0.0,0.0,495.64


In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df['TIPO GASTO']= df['TIPO GASTO'].replace(['COMPRAS '], "COMPRAS")

In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['TIPO GASTO'])
unico

array(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'BOTIQUIN', 'CALIBRACIONES', 'CAPACITACION', 'COMBUSTIBLE ',
       'COMISION BANCARIA', 'COMISION MIXTA', 'COMPRA COMPUTADORA ',
       'COMPRA TABLET ', 'COMPRAS', 'DERECHOS', 'ENERGIA ELECTRICA',
       'FLETE', 'FLETES', 'GASOLINA', 'GASTOS IMPORTACION ',
       'GERENCIA ADMON', 'GERENCIA VTAS', 'HONORARIOS CONTABLES',
       'HONORARIOS JURIDICOS', 'HONORARIOS PF', 'HONORARIOS PM',
       'HOSPEDAJE', 'HOSPEDAJE ', 'IMSS/INFONAVIT', 'MAQUILAS',
       'MAQUILAS GIC', 'MAQUINARIA', 'MEDICOS', 'MENSAJERIA',
       'MONITOREO CAMARAS ADT', 'MTTO COMPUTO', 'MTTO LOCAL',
       'MTTO MAQUINARIA', 'MTTO OFICINA ', 'MTTO TRANSPORTE',
       'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'RENTA COPIADORA', 'RENTA OFICINA ', 'SANITIZACION', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUELDOS PRODUCCION', 'SUELDOS VENTAS',
       'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE', 'UNIFORME',
      

In [ ]:
#Convierto una variable a dicotómica 
df['TIPO GASTO']= df['TIPO GASTO'].replace(['ALIMENTOS', 'ARRENDAMIENTO FINANCIERO', 'ARRENDAMIENTO LOCAL',
       'BOTIQUIN', 'CALIBRACIONES', 'CAPACITACION', 'COMBUSTIBLE ',
       'COMISION BANCARIA', 'COMISION MIXTA', 'COMPRA COMPUTADORA ',
       'COMPRA TABLET ', 'DERECHOS',
       'ENERGIA ELECTRICA', 'FLETE', 'FLETES', 'GASOLINA',
       'GASTOS IMPORTACION ', 'GERENCIA ADMON', 'GERENCIA VTAS',
       'HONORARIOS CONTABLES', 'HONORARIOS JURIDICOS', 'HONORARIOS PF',
       'HONORARIOS PM', 'HOSPEDAJE', 'HOSPEDAJE ', 'IMSS/INFONAVIT',
       'MAQUILAS', 'MAQUILAS GIC', 'MAQUINARIA', 'MEDICOS', 'MENSAJERIA',
       'MONITOREO CAMARAS ADT', 'MTTO COMPUTO', 'MTTO LOCAL',
       'MTTO MAQUINARIA', 'MTTO OFICINA ', 'MTTO TRANSPORTE',
       'NO DEDUCIBLE', 'PAPELERIA', 'PAQUETERIA', 'PEAJES', 'PUBLICIDAD',
       'RENTA COPIADORA', 'RENTA OFICINA ', 'SANITIZACION', 'SEGUROS',
       'SUELDOS DIRECCION', 'SUELDOS PRODUCCION', 'SUELDOS VENTAS',
       'SUSCRIPCIONES', 'TELEFONO', 'TRANSPORTE', 'UNIFORME',
       'VALES DESPENSA'], "OTROS")

In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['TOTAL MX']]
Var_Dep= df['TIPO GASTO']

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['COMPRAS', 'COMPRAS', 'OTROS', 'COMPRAS', 'OTROS', 'COMPRAS',
       'OTROS', 'OTROS', 'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'COMPRAS',
       'OTROS', 'COMPRAS', 'OTROS', 'COMPRAS', 'OTROS', 'COMPRAS',
       'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'OTROS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'OTROS', 'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'COMPRAS',
       'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'COMPRAS', 'OTROS',
       'COMPRAS', 'OTROS', 'COMPRAS', 'COMPRAS', 'OTROS', 'COMPRAS',
       'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'OTROS',
       'OTROS', 'COMPRAS', 'OTROS', 'COMPRAS', 'COMPRAS', 'OTROS',
       'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS',
       'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS', 'OTROS',
       'COMPRAS', 'COMPRAS', 'OTROS', 'OTROS', 'OTROS', 'COMPRAS',
       'COMPRAS', 'OTROS', 'COMPRAS', 'COMPRAS', 'COMPRAS', 'COMPRAS',
       'COMPRAS', 'COMPRAS',

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[226 220]
 [195 270]]


In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="weighted", pos_label="COMPRAS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.5440668686601126


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'COMPRAS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.544456641053787


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="weighted", pos_label="OTROS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.544456641053787


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'OTROS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="weighted", pos_label="COMPRAS")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.543851439637287


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'COMPRAS') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
